# Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC, wait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
import requests
import os
import sys
import pyodbc

# Download CSV File

In [2]:
# open covid data website
path = r"C:\Users\Abdulkadir\Documents\Programming\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get('https://ourworldindata.org/covid-deaths')
wait = WebDriverWait(driver, 5)


# download csv file
try:
    csvLink = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/main/article/div[3]/div[2]/div/div/section[1]/div/div[1]/div[1]/div/div/a[2]")))
    csvLink.click()
except:
    print('Download failed')
    sys.exit()

    
# file path of csv
filepath = r"C:\Users\Abdulkadir\Downloads\owid-covid-data.csv"


# load csv file into df
x = 1
while True:
    try:
        csv = pd.read_csv(filepath)
        print('CSV succesfully downloaded.')
        break
    except:
        print(f'CSV downloading... {x}/100.')
        x += 1
        time.sleep(3)
    
    if x == 30:
        print('CSV Failed')
        #sys.exit()

# close driver
driver.close()
        
# delete the csv file off my computer so it does not interfere with downlaoding new data and names
try:
    os.remove(filepath)
    print('CSV successfully deleted.')
except:
    print('CSV does not exist.')

CSV downloading... 1/100.
CSV downloading... 2/100.
CSV downloading... 3/100.
CSV downloading... 4/100.
CSV downloading... 5/100.
CSV downloading... 6/100.
CSV succesfully downloaded.
CSV successfully deleted.


# Take Relevant Data

In [3]:
data = csv[['continent', 'location', 'date', 'population', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_vaccinations', 'new_vaccinations']]

# make NaN into 0
data = data.fillna(0)

In [4]:
data.tail()

,continent,location,date,population,total_cases,new_cases,total_deaths,new_deaths,total_vaccinations,new_vaccinations
176910,Africa,Zimbabwe,2022-04-02,15092171.0,246481.0,67.0,5446.0,2.0,9600524.0,119330.0
176911,Africa,Zimbabwe,2022-04-03,15092171.0,246525.0,44.0,5446.0,0.0,9694465.0,93941.0
176912,Africa,Zimbabwe,2022-04-04,15092171.0,246612.0,87.0,5451.0,5.0,9770202.0,75737.0
176913,Africa,Zimbabwe,2022-04-05,15092171.0,246744.0,132.0,5451.0,0.0,9807529.0,37327.0
176914,Africa,Zimbabwe,2022-04-06,15092171.0,246803.0,59.0,5453.0,2.0,9859654.0,52125.0


# Move Data to SQL

In [5]:
SERVER_NAME = 'DESKTOP-GR6T3L7'
DATABASE_NAME = 'Covid Project'

# connection string
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; \
                       SERVER=' + SERVER_NAME + '; \
                       DATABASE=' + DATABASE_NAME + '; \
                       Trusted_Connection=yes')

# cursor to extract data
cursor = conn.cursor()

# clear SQL table for new data
cursor.execute('DELETE FROM [Covid Project].[dbo].[Automation]')
print('Table cleared from SQL')

# insert new data into table
for index, row in data.iterrows():
    cursor.execute("INSERT INTO [Covid Project].[dbo].[Automation] (continent,location,date,population,total_cases,new_cases,total_deaths,new_deaths,total_vaccinations,new_vaccinations) values(?,?,?,?,?,?,?,?,?,?)", row.continent, row.location, row.date, row.population, row.total_cases, row.new_cases, row.total_deaths, row.new_deaths, row.total_vaccinations, row.new_vaccinations)
cursor.commit()
print('New data uploaded to SQL')

# close connection adn remove cursor
cursor.close()
conn.close()

Table cleared from SQL
New data uploaded to SQL


# Analysis using Pandas

------------------------- Deaths per cases in the United Kingdom -------------------------

In [6]:
# Finding the percentage of total deaths per total case for each day in the uk
DeathsPerCase = (data['total_deaths'] / data['total_cases']) * 100

# make it into df
DeathsPerCase = pd.DataFrame(DeathsPerCase, columns=['deaths_per_cases'])

# data needed
table_1 = data[['location', 'date', 'total_cases', 'total_deaths']]

# combines the new variable column to table 1
first_analysis = table_1.join(DeathsPerCase)

# filter data for only location in united kingdom
is_UK = first_analysis['location'] == 'United Kingdom'
first_analysis = first_analysis[is_UK]
first_analysis

,location,date,total_cases,total_deaths,deaths_per_cases
165860,United Kingdom,2020-01-31,2.0,0.0,0.000000
165861,United Kingdom,2020-02-01,2.0,0.0,0.000000
165862,United Kingdom,2020-02-02,2.0,0.0,0.000000
165863,United Kingdom,2020-02-03,8.0,0.0,0.000000
165864,United Kingdom,2020-02-04,8.0,0.0,0.000000
...,...,...,...,...,...
166652,United Kingdom,2022-04-02,21285727.0,165721.0,0.778555
166653,United Kingdom,2022-04-03,21285727.0,165721.0,0.778555
166654,United Kingdom,2022-04-04,21428717.0,165931.0,0.774339
166655,United Kingdom,2022-04-05,21479718.0,166304.0,0.774237


In [ ]:
# line graph

-------------------- Percentage of total cases in the United Kingdom vs Time --------------------

In [7]:
# total cases per popualtion variable
TotCasesPerPopPerc = (data['total_cases'] / data['population']) * 100
TotCasesPerPopPerc = pd.DataFrame(TotCasesPerPopPerc, columns=['total_cases_per_population_percentage'])

second_analysis = data[['location', 'date']].join(TotCasesPerPopPerc)
second_analysis = second_analysis[is_UK]
second_analysis

,location,date,total_cases_per_population_percentage
165860,United Kingdom,2020-01-31,0.000003
165861,United Kingdom,2020-02-01,0.000003
165862,United Kingdom,2020-02-02,0.000003
165863,United Kingdom,2020-02-03,0.000012
165864,United Kingdom,2020-02-04,0.000012
...,...,...,...
166652,United Kingdom,2022-04-02,31.207488
166653,United Kingdom,2022-04-03,31.207488
166654,United Kingdom,2022-04-04,31.417129
166655,United Kingdom,2022-04-05,31.491903


In [ ]:
# line graph

-------------------- Death percentage in the world -------------------------

In [8]:
# relevant data
third_analysis = data[['continent', 'location', 'new_cases', 'new_deaths']]
third_analysis

# group by location and continent
is_grouped = third_analysis.groupby(['location', 'continent']).sum()
is_grouped.rename(columns={"new_cases":"total_cases", "new_deaths":"total_deaths"}, inplace=True)

# death percentage
deathPercentage = (is_grouped['total_deaths'] / is_grouped['total_cases']) * 100
deathPercentage = pd.DataFrame(deathPercentage, columns=['death_percentage'])

# combine death percentage to grouped table
combined_table = is_grouped.join(deathPercentage)

# remove NaN
combined_table['death_percentage'] = combined_table['death_percentage'].fillna(0)

# remove continent with values of 0 to get rid of continents and only leave countries
death_percentage_table = combined_table.query("continent!=0")

# remove death percentage
death_percentage_table = death_percentage_table[death_percentage_table['death_percentage'] > 0]

# sort table by total_deaths desc
death_percentage_table.sort_values(by='total_deaths', ascending=False, inplace=True)
death_percentage_table

,,total_cases,total_deaths,death_percentage
location,continent,,,
United States,North America,80248985.0,986280.0,1.229025
Brazil,South America,29964012.0,660980.0,2.205913
India,Asia,43031958.0,514157.0,1.194826
Russia,Europe,17679300.0,363175.0,2.054239
Mexico,North America,5683288.0,316342.0,5.566179
...,...,...,...,...
Palau,Oceania,4070.0,6.0,0.147420
Vanuatu,Oceania,4713.0,3.0,0.063654
Montserrat,North America,175.0,3.0,1.714286


In [ ]:
# try plot map of world with colours

In [9]:
# print table to have a look
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(death_percentage_table)

-------------------- Infection rate by looking at total cases against population -------------------------

In [117]:
# Relevant data
fourth_analysis = data[['continent', 'location', 'population', 'total_cases']]

# Infection rate: Max total cases / population, group by continent location population
is_grouped_2 = fourth_analysis.groupby(['location', 'continent']).max()

# Infection rate
infection_rate = (is_grouped_2['total_cases'] / is_grouped_2['population'])*100
infection_rate = pd.DataFrame(infection_rate, columns=['infection_rate'])

# Combine the column with the table
combined_table_2 = is_grouped_2.join(infection_rate)
combined_table_2.sort_values(by=['infection_rate'], ascending=False, inplace=True)

# remove continent with values of 0 to get rid of continents and only leave countries
combined_table_2 = combined_table_2.query("continent!=0 and total_cases!=0")

# remove NaN
combined_table_2['infection_rate'] = combined_table_2['infection_rate'].fillna(0)
combined_table_2

,,population,total_cases,infection_rate
location,continent,,,
Faeroe Islands,Europe,4.905300e+04,34658.0,70.654190
Denmark,Europe,5.813302e+06,3075548.0,52.905354
Andorra,Europe,7.735400e+04,40024.0,51.741345
Gibraltar,Europe,3.369100e+04,17188.0,51.016592
Cyprus,Europe,8.960050e+05,449877.0,50.209206
...,...,...,...,...
Niger,Africa,2.513081e+07,8822.0,0.035104
China,Asia,1.444216e+09,314707.0,0.021791
Macao,Asia,6.583910e+05,82.0,0.012455


In [ ]:
# another world map

-------------------- Death percentage for each country --------------------

In [109]:
# Relevant data#
fifth_analysis = data[['location', 'continent', 'population', 'total_deaths']]
fifth_analysis.head(50)

# Group data
is_grouped_3 = fifth_analysis.groupby(['location', 'continent']).max()

# Total deaths per population
deaths_per_population = (is_grouped_3['total_deaths'] / is_grouped_3['population'])*100
deaths_per_population = pd.DataFrame(deaths_per_population, columns=['deaths_per_population_percentage'])

# Join deaths per population to table
combined_table_3 = is_grouped_3.join(deaths_per_population)
combined_table_3.sort_values(by=['total_deaths'], ascending=False, inplace=True)

# remove continent with values of 0 to get rid of continents and only leave countries
combined_table_3 = combined_table_3.query("continent!=0 and total_deaths!=0")

# remove NaN
combined_table_3['deaths_per_population_percentage'] = combined_table_3['deaths_per_population_percentage'].fillna(0)
combined_table_3

,,population,total_deaths,deaths_per_population_percentage
location,continent,,,
United States,North America,3.329151e+08,983817.0,0.295516
Brazil,South America,2.139934e+08,660980.0,0.308879
India,Asia,1.393409e+09,521530.0,0.037428
Russia,Europe,1.459120e+08,363175.0,0.248900
Mexico,North America,1.302622e+08,323403.0,0.248271
...,...,...,...,...
Palau,Oceania,1.817400e+04,6.0,0.033014
Vanuatu,Oceania,3.144640e+05,3.0,0.000954
Montserrat,North America,4.981000e+03,3.0,0.060229


In [ ]:
# another world map

-------------------- Continental Death Count --------------------

In [48]:
# relevant data
sixth_analysis = data[['location', 'continent', 'total_deaths']]

# group data
is_grouped_5 = sixth_analysis.groupby(['location', 'continent']).max()

# keep continents only
is_grouped_5 = is_grouped_5.query("continent==0")
is_grouped_5.sort_values(by=['total_deaths'], ascending=False, inplace=True)

# only keep europe, north america, asia, south america, africa, oceania
is_grouped_5 = is_grouped_5.query("(location=='Europe') | (location=='North America') | (location=='Asia') | (location=='South America') | (location=='Africa') | (location=='Oceania')")
is_grouped_5

,,total_deaths
location,continent,
Europe,0,1784440.0
North America,0,1418071.0
Asia,0,1412726.0
South America,0,1289522.0
Africa,0,252202.0
Oceania,0,9534.0


-------------------- World Death percentage vs Date --------------------

In [112]:
# relevant data
seventh_analysis = data[['location', 'date', 'new_cases', 'new_deaths']]

# world data
world_data = seventh_analysis.query("(location=='World') and (new_cases!=0)")
world_data

,location,date,new_cases,new_deaths
173885,World,2020-01-23,100.0,1.0
173886,World,2020-01-24,287.0,8.0
173887,World,2020-01-25,493.0,16.0
173888,World,2020-01-26,683.0,14.0
173889,World,2020-01-27,809.0,26.0
...,...,...,...,...
174685,World,2022-04-02,1043688.0,2661.0
174686,World,2022-04-03,789065.0,1643.0
174687,World,2022-04-04,900535.0,2926.0
174688,World,2022-04-05,1335628.0,4011.0


In [ ]:
# line graph

-------------------- World Cases and Deaths --------------------

In [76]:
grouped_world_data = world_data.groupby(['location']).sum()
grouped_world_data.rename(columns={"new_cases":"total_cases", "new_deaths":"total_deaths"}, inplace=True)

# death percentage
world_death_percentage = (grouped_world_data['total_deaths'] / grouped_world_data['total_cases'])*100
world_death_percentage = pd.DataFrame(world_death_percentage, columns=['death_percentage'])

# join two tables
world_data_final = grouped_world_data.join(world_death_percentage)
world_data_final

,total_cases,total_deaths,death_percentage
location,,,
World,493971830.0,6132310.0,1.241429


-------------------- Population and Vaccinations --------------------

In [94]:
# relevant data
eighth_analysis = data[['location', 'continent', 'date', 'population', 'total_vaccinations']]
eighth_analysis

# group by location
is_grouped_6 = eighth_analysis.groupby(['location', 'continent']).max()
is_grouped_6

# vaccination percentage
vaccination_percentage = (is_grouped_6['total_vaccinations'] / is_grouped_6['population'])*100
vaccination_percentage = pd.DataFrame(vaccination_percentage, columns=['vaccination_percentage'])

# join tables
combined_4 = is_grouped_6.join(vaccination_percentage)
combined_4

# keep continents only
combined_4 = combined_4.query("continent!=0 and population != 0 and total_vaccinations != 0")
combined_4.sort_values(by=['vaccination_percentage'], ascending=False, inplace=True)
combined_4

,,date,population,total_vaccinations,vaccination_percentage
location,continent,,,,
Gibraltar,Europe,2022-04-06,33691.0,117937.0,350.054911
Cuba,North America,2022-04-06,11317498.0,35457646.0,313.299335
Chile,South America,2022-04-06,19212362.0,50998553.0,265.446555
Niue,Oceania,2022-03-13,1614.0,4161.0,257.806691
Singapore,Asia,2022-04-06,5453600.0,13864639.0,254.229115
...,...,...,...,...,...
South Sudan,Africa,2022-04-06,11381377.0,575057.0,5.052614
Yemen,Asia,2022-04-06,30490639.0,807502.0,2.648360
Haiti,North America,2022-04-06,11541683.0,253807.0,2.199047


In [ ]:
# world map

#  Same analysis using SQL Queries 

In [99]:
# connection string
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; \
                       SERVER=' + SERVER_NAME + '; \
                       DATABASE=' + DATABASE_NAME + '; \
                       Trusted_Connection=yes')

# cursor to extract data
cursor = conn.cursor()

------------------------- Deaths per cases in the United Kingdom -------------------------

In [103]:
query_1 = """ SELECT location, date, total_cases, total_deaths, (total_deaths/total_cases * 100) AS DeathsPerCase
              FROM [Covid Project].dbo.[Automation]
              WHERE location = 'United Kingdom'
              ORDER BY 1, 2 """

sql_result_1 = pd.read_sql(query_1, conn)
sql_result_1

,location,date,total_cases,total_deaths,DeathsPerCase
0,United Kingdom,2020-01-31,2.0,0.0,0.000000
1,United Kingdom,2020-02-01,2.0,0.0,0.000000
2,United Kingdom,2020-02-02,2.0,0.0,0.000000
3,United Kingdom,2020-02-03,8.0,0.0,0.000000
4,United Kingdom,2020-02-04,8.0,0.0,0.000000
...,...,...,...,...,...
792,United Kingdom,2022-04-02,21285727.0,165721.0,0.778555
793,United Kingdom,2022-04-03,21285727.0,165721.0,0.778555
794,United Kingdom,2022-04-04,21428717.0,165931.0,0.774339
795,United Kingdom,2022-04-05,21479718.0,166304.0,0.774237


-------------------- Percentage of total cases in the United Kingdom vs Time --------------------

In [104]:
query_2 = """ SELECT location, date, (total_cases/population * 100) AS TotalCasesPerPopulationPercentage
              FROM [Covid Project].dbo.[Automation]
              WHERE location = 'United Kingdom'
              ORDER BY 2 """

sql_result_2 = pd.read_sql(query_2, conn)
sql_result_2

,location,date,TotalCasesPerPopulationPercentage
0,United Kingdom,2020-01-31,0.000003
1,United Kingdom,2020-02-01,0.000003
2,United Kingdom,2020-02-02,0.000003
3,United Kingdom,2020-02-03,0.000012
4,United Kingdom,2020-02-04,0.000012
...,...,...,...
792,United Kingdom,2022-04-02,31.207488
793,United Kingdom,2022-04-03,31.207488
794,United Kingdom,2022-04-04,31.417129
795,United Kingdom,2022-04-05,31.491903


-------------------- Death percentage in the world -------------------------

In [105]:
query_3 = """ SELECT location, SUM(new_cases) AS TotalCases, SUM(new_deaths) AS TotalDeaths, ROUND((SUM(new_deaths)/SUM(new_cases))*100, 2) AS DeathPercentage
              FROM [Covid Project].dbo.[Automation]
              WHERE continent <> '0' -- gets rid of the continents and limits to countries
              GROUP BY location
              HAVING SUM(new_cases) > 0 AND  SUM(new_deaths) > 0 -- gets rid of NULL
              ORDER BY 3 DESC """

sql_result_3 = pd.read_sql(query_3, conn)
sql_result_3

,location,TotalCases,TotalDeaths,DeathPercentage
0,United States,80248985.0,986280.0,1.23
1,Brazil,29964012.0,660980.0,2.21
2,India,43031958.0,514157.0,1.19
3,Russia,17679300.0,363175.0,2.05
4,Mexico,5683288.0,316342.0,5.57
...,...,...,...,...
203,Palau,4070.0,6.0,0.15
204,Vanuatu,4713.0,3.0,0.06
205,Montserrat,175.0,3.0,1.71
206,Samoa,2782.0,2.0,0.07


-------------------- Infection rate by looking at total cases against population -------------------------

In [107]:
query_4 = """ SELECT location, MAX(total_cases) as MaximumCases, population, (MAX(total_cases)/population)*100 AS InfectionRate
              FROM [Covid Project].dbo.[Automation]
              WHERE continent <> '0' -- gets rid of the continents and limits to countries
              GROUP BY location, population
              HAVING SUM(new_cases) > 0 AND  SUM(CAST(new_deaths as int)) > 0 -- gets rid of NULL
              ORDER BY 4 DESC """

sql_result_4 = pd.read_sql(query_4, conn)
sql_result_4

,location,MaximumCases,population,InfectionRate
0,Faeroe Islands,34658.0,4.905300e+04,70.654190
1,Denmark,3075548.0,5.813302e+06,52.905354
2,Andorra,40024.0,7.735400e+04,51.741345
3,Gibraltar,17188.0,3.369100e+04,51.016592
4,Cyprus,449877.0,8.960050e+05,50.209206
...,...,...,...,...
203,Tanzania,33815.0,6.149844e+07,0.054985
204,Chad,7349.0,1.691498e+07,0.043447
205,Yemen,11812.0,3.049064e+07,0.038740
206,Niger,8822.0,2.513081e+07,0.035104


-------------------- Death percentage for each country --------------------

In [108]:
query_5 = """ SELECT location, MAX(CAST(total_deaths AS int)) as TotalDeathsUpToDate, population, (MAX(CAST(total_deaths AS int))/population)*100 AS TotalDeathPerPopulation
              FROM [Covid Project].dbo.[Automation]
              WHERE continent <> '0' -- gets rid of the continents and limits to countries
              GROUP BY location, population
              HAVING MAX(total_cases) > 0 AND MAX(CAST(total_deaths as int)) > 0 -- gets rid of NULL
              ORDER BY 2 DESC """

sql_result_5 = pd.read_sql(query_5, conn)
sql_result_5

,location,TotalDeathsUpToDate,population,TotalDeathPerPopulation
0,United States,983817,3.329151e+08,0.295516
1,Brazil,660980,2.139934e+08,0.308879
2,India,521530,1.393409e+09,0.037428
3,Russia,363175,1.459120e+08,0.248900
4,Mexico,323403,1.302622e+08,0.248271
...,...,...,...,...
203,Palau,6,1.817400e+04,0.033014
204,Vanuatu,3,3.144640e+05,0.000954
205,Montserrat,3,4.981000e+03,0.060229
206,Samoa,2,2.001440e+05,0.000999


-------------------- Continental Death Count --------------------

In [110]:
query_6 = """ SELECT location, MAX(CAST(total_deaths AS int)) AS TotalDeathCount
              FROM [Covid Project].dbo.Automation
              WHERE continent = '0' AND location NOT LIKE '%income%' AND location NOT LIKE '%European%' AND location NOT LIKE '%International%' AND location NOT LIKE '%World%'
              GROUP BY location
              ORDER BY 2 DESC """

sql_result_6 = pd.read_sql(query_6, conn)
sql_result_6

,location,TotalDeathCount
0,Europe,1784440
1,North America,1418071
2,Asia,1412726
3,South America,1289522
4,Africa,252202
5,Oceania,9534


-------------------- World Death percentage vs Date --------------------

In [111]:
query_7 = """ SELECT date, new_cases, new_deaths, ( new_deaths / new_cases)*100 AS DeathPercentagePerInfected
              FROM [Covid Project].dbo.Automation
              WHERE location = 'World' AND new_cases <> '0' """

sql_result_7 = pd.read_sql(query_7, conn)
sql_result_7

,date,new_cases,new_deaths,DeathPercentagePerInfected
0,2020-01-23,100.0,1.0,1.000000
1,2020-01-24,287.0,8.0,2.787456
2,2020-01-25,493.0,16.0,3.245436
3,2020-01-26,683.0,14.0,2.049780
4,2020-01-27,809.0,26.0,3.213844
...,...,...,...,...
800,2022-04-02,1043688.0,2661.0,0.254961
801,2022-04-03,789065.0,1643.0,0.208221
802,2022-04-04,900535.0,2926.0,0.324918
803,2022-04-05,1335628.0,4011.0,0.300308


-------------------- World Cases and Deaths --------------------

In [113]:
query_8 = """ SELECT SUM(new_cases) AS TotalCases, SUM(CAST(new_deaths AS int)) AS TotalDeaths, (SUM(CAST(new_deaths AS int))/SUM(new_cases))*100 AS DeathPercentage
              FROM [Covid Project].dbo.Automation
              WHERE continent  <> '0' """

sql_result_8 = pd.read_sql(query_8, conn)
sql_result_8

,TotalCases,TotalDeaths,DeathPercentage
0,493971109.0,6132295,1.241428


-------------------- Population and Vaccinations --------------------

In [115]:
query_9 = """ SELECT location, population, MAX(total_vaccinations) as total_vaccinations, (MAX(total_vaccinations)/population)*100 AS vaccination_percentage
              FROM [Covid Project].dbo.Automation
              WHERE continent <> '0' AND total_vaccinations <> '0' AND population <> '0'
              GROUP BY location, population
              ORDER BY 4 DESC """

sql_result_9 = pd.read_sql(query_9, conn)
sql_result_9

,location,population,total_vaccinations,vaccination_percentage
0,Gibraltar,33691.0,117937.0,350.054911
1,Cuba,11317498.0,35457646.0,313.299335
2,Chile,19212362.0,50998553.0,265.446555
3,Niue,1614.0,4161.0,257.806691
4,Singapore,5453600.0,13864639.0,254.229115
...,...,...,...,...
213,South Sudan,11381377.0,575057.0,5.052614
214,Yemen,30490639.0,807502.0,2.648360
215,Haiti,11541683.0,253807.0,2.199047
216,Democratic Republic of Congo,92377986.0,964948.0,1.044565


In [116]:
# close connection and remove cursor
cursor.close()
conn.close()